### import libraries 

In [1]:
import tkinter as tk
from tkinter import ttk
from tkinter import Label, Entry

import pandas as pd
import numpy as np
import pickle
import joblib
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler
    
from sklearn.impute import SimpleImputer
from keras.models import model_from_json
import warnings
warnings.filterwarnings('ignore')

### main application window

In [2]:
import tkinter as tk
from tkinter import ttk
import pandas as pd
import joblib
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.decomposition import PCA
from tensorflow.keras.models import model_from_json

# Define label mappings for features that use LabelEncoder
label_mapping = {
    "Gender": ["Male", "Female"],
    "SeniorCitizen": ["Yes", "No"],
    "Partner": ["Yes", "No"],
    "Dependents": ["Yes", "No"],
    "PhoneService": ["Yes", "No"],
    "MultipleLines": ["Yes", "No"],
    "InternetService": ['DSL', 'Fiber optic', 'No'],
    "OnlineSecurity": ["Yes", "No"],
    "OnlineBackup": ["Yes", "No"],
    "DeviceProtection": ["Yes", "No"],
    "TechSupport": ["Yes", "No"],
    "StreamingTV": ["Yes", "No"],
    "StreamingMovies": ["Yes", "No"],
    "Contract": ['Month-to-month', 'Two year', 'One year'],
    "PaperlessBilling": ["Yes", "No"],
    "PaymentMethod": ['Electronic check', 'Mailed check', 'Bank transfer (automatic)', 'Credit card (automatic)']
}

# Create the main application window
window = tk.Tk()
window.geometry("480x700")
window.title("Churn Prediction App")

# Create a vertical scrollbar
scrollbar = tk.Scrollbar(window)
scrollbar.pack(side="right", fill="y")

# Create a canvas to contain the elements
canvas = tk.Canvas(window, yscrollcommand=scrollbar.set)
canvas.pack(fill="both", expand=True)

# Configure the scrollbar to work with the canvas
scrollbar.config(command=canvas.yview)

# Create a frame to hold the elements
frame = tk.Frame(canvas)
canvas.create_window((0, 0), window=frame, anchor="nw")

# Labels for displaying predictions
classLabel = tk.Label(frame, height=2, width=20, font=("Arial", 14))
classLabel.grid(row=0, column=0, columnspan=2)
classLabel.configure(text='Churn Prediction')

# Create a label widget for displaying the prediction result
resultLabel = Label(window, text="", font=("Helvetica", 16))
resultLabel.pack()

# Create input fields for features
feature_labels = [
    "Gender", "SeniorCitizen", "Partner", "Dependents", "Tenure", "PhoneService", "MultipleLines",
    "InternetService", "OnlineSecurity", "OnlineBackup", "DeviceProtection", "TechSupport", "StreamingTV",
    "StreamingMovies", "Contract", "PaperlessBilling", "PaymentMethod", "MonthlyCharges", "TotalCharges"
]

entry_fields = {}
dropdown_fields = {}

# Initialize userSelection with empty strings
userSelection = [""] * len(feature_labels)
dropdown_lists = []
numerical_fields = []

# Initialize the total_payment variable
total_payment = 0.0
global user_selection_df

feature_options = {
    "Gender": ["Male", "Female"],
    "SeniorCitizen": ["Yes", "No"],
    "Partner": ["Yes", "No"],
    "Dependents": ["Yes", "No"],
    "PhoneService": ["Yes", "No"],
    "MultipleLines": ["Yes", "No"],
    "InternetService": ['DSL', 'Fiber optic', 'No'],
    "OnlineSecurity": ["Yes", "No"],
    "OnlineBackup": ["Yes", "No"],
    "DeviceProtection": ["Yes", "No"],
    "TechSupport": ["Yes", "No"],
    "StreamingTV": ["Yes", "No"],
    "StreamingMovies": ["Yes", "No"],
    "Contract": ['Month-to-month', 'Two year', 'One year'],
    "PaperlessBilling": ["Yes", "No"],
    "PaymentMethod": ['Electronic check', 'Mailed check', 'Bank transfer (automatic)', 'Credit card (automatic)']
}

column_names = [
    "Gender", "SeniorCitizen", "Partner", "Dependents", "Tenure", "PhoneService", "MultipleLines",
    "InternetService", "OnlineSecurity", "OnlineBackup", "DeviceProtection", "TechSupport", "StreamingTV",
    "StreamingMovies", "Contract", "PaperlessBilling", "PaymentMethod", "MonthlyCharges", "TotalCharges",
    "TotalPayment",
    "HasOnlineSecurityBackup", "HasTechSupportAndDeviceProtection",
    "MonthlyToTotalChargesRatio", "AdditionalServices", "NEW_noProt",
    "NEW_AVG_Charges", "NEW_Increase", "NEW_AVG_Service_Fee", "Churn"
]

# Function to update userSelection when an entry field loses focus
def update_user_selection_entry(event, label):
    entry_value = event.widget.get()
    if entry_value.strip():  # Check if the input is not empty
        userSelection[feature_labels.index(label)] = float(entry_value)
    else:
        userSelection[feature_labels.index(label)] = ""

for idx, feature_label in enumerate(feature_labels):
    tk.Label(frame, text=feature_label + ":").grid(row=3 + idx, column=0, pady=(5, 5))

    # Create input fields for numerical features
    if feature_label in ["MonthlyCharges", "TotalCharges", "Tenure"]:
        entry_fields[feature_label] = tk.Entry(frame, font=("Arial", 12))
        entry_fields[feature_label].grid(row=3 + idx, column=1, pady=(5, 5))
        entry_fields[feature_label].bind("<FocusOut>", lambda event, label=feature_label: update_user_selection_entry(event, label))
        numerical_fields.append(entry_fields[feature_label])

    # Create a dropdown list for features with predefined options
    elif feature_label in feature_options:
        feature_values = feature_options[feature_label]
        feature_var = tk.StringVar()
        feature_combobox = ttk.Combobox(frame, textvariable=feature_var, values=feature_values)
        feature_var.trace_add("write", lambda *args, var=feature_var, label=feature_label: update_user_selection(var, label))
        feature_combobox.grid(row=3 + idx, column=1, columnspan=3, pady=(5, 5))
        dropdown_fields[feature_label] = feature_combobox
        dropdown_lists.append(feature_label)

def update_user_selection(var, label):
    selected_value = var.get()
    userSelection[feature_labels.index(label)] = selected_value


def predict_and_display_result():
    global userSelection  # Declare userSelection as a global variable
    global user_selection_df, num_features, X_cat, X_num, X

    # Ensure userSelection has the same number of elements as feature_labels
    while len(userSelection) < len(feature_labels):
        userSelection.append("")  # Fill with empty strings for missing values

    # Check if fields with numeric values are not empty before performing calculations
    try:
        tenure = float(userSelection[feature_labels.index("Tenure")])
        monthly_charges = float(userSelection[feature_labels.index("MonthlyCharges")])
    except ValueError:
        resultLabel.configure(text='Please enter valid numeric values for Tenure and MonthlyCharges.')
        return  # Exit the function if the input is not valid

    # Save userSelection before labeling in append mode
    user_selection_before_labeling_df = pd.DataFrame([userSelection], columns=feature_labels)
    user_selection_before_labeling_df.to_csv('user_selection_before_labeling.csv', mode='a', header=False, index=False)

    le = LabelEncoder()

    for feature in dropdown_lists:
        labels = label_mapping[feature]
        le.fit(labels)
        transformed_value = le.transform([userSelection[feature_labels.index(feature)]])[0]
        transformed_values.append(transformed_value)  # Append transformed value to the list
        userSelection[feature_labels.index(feature)] = transformed_value

    print("Transformed values as an array:", transformed_values)  # Print the list of transformed values as an array
    print("userSelection after mapping:", userSelection)

    # Total Charges
    userSelection[-1] = userSelection[4] * float(userSelection[17])

    # HasOnlineSecurityBackup
    userSelection.append(userSelection[8] & userSelection[9])

    # HasTechSupportAndDeviceProtection
    userSelection.append(userSelection[10] & userSelection[11])

    # MonthlyToTotalChargesRatio
    userSelection.append(userSelection[17] / userSelection[18])

    # AdditionalServices (sum of OnlineSecurity, OnlineBackup, DeviceProtection, TechSupport, StreamingTV, StreamingMovies)
    additional_services = userSelection[8] + userSelection[9] + userSelection[10] + userSelection[11] + userSelection[12] + userSelection[13]
    userSelection.append(additional_services)

    # NEW_noProt (1 if none of OnlineBackup, DeviceProtection, TechSupport are 1, else 0)
    new_no_prot = 1 if (userSelection[9] != 1) or (userSelection[10] != 1) or (userSelection[11] != 1) else 0
    userSelection.append(new_no_prot)

    # NEW_AVG_Charges (TotalCharges / (Tenure + 1))
    new_avg_charges = userSelection[18] / (userSelection[4] + 1)
    userSelection.append(new_avg_charges)

    # NEW_Increase (NEW_AVG_Charges / MonthlyCharges)
    new_increase = new_avg_charges / userSelection[17]
    userSelection.append(new_increase)

    # NEW_AVG_Service_Fee (MonthlyCharges / (AdditionalServices + 1))
    new_avg_service_fee = userSelection[17] / (additional_services + 1)
    userSelection.append(new_avg_service_fee)

    # Total Payment
    total_payment = userSelection[4] * float(userSelection[17])
    userSelection.append(total_payment)

    # churn
    churn = 0
    userSelection.append(churn)
    print("userSelection after edit:", userSelection)
    user_selection_df = pd.DataFrame([userSelection], columns=column_names)

    print("Length of feature_labels:", len(feature_labels))
    print("Length of userSelection:", len(userSelection))
    print("Feature labels:", feature_labels)
    print("User selection:", userSelection)

    user_selection_df.to_csv('user_selection_tinker.csv', mode='a', header=False, index=False)

    user_selection_df = pd.DataFrame([userSelection], columns=column_names)

    print("Length of feature_labels:", len(feature_labels))
    print("Length of userSelection:", len(userSelection))
    print("Feature labels:", feature_labels)
    print("User selection:", userSelection)

    # Load your dataset from user_selection_tinker
    dataset = pd.read_csv('user_selection_tinker.csv')

    # Extract the last row (assuming the last row is at index -1)
    last_row = dataset.iloc[-1]

    print(user_selection_df.columns)

    num_features = [
        'Tenure', 'MonthlyCharges', 'TotalCharges', 'MonthlyToTotalChargesRatio',
        'TotalPayment', 'NEW_AVG_Charges', 'NEW_Increase', 'NEW_AVG_Service_Fee',
        'HasOnlineSecurityBackup', 'HasTechSupportAndDeviceProtection', 'AdditionalServices'
    ]

    X_cat = last_row.drop(columns=num_features)
    X_num = last_row[num_features].values.reshape(1, -1)

    print("X_num shape:", X_num.shape)  # Print the shape of X_num for debugging

    # Scale the numerical features
    scaler = MinMaxScaler()
    X_num_scaled = scaler.fit_transform(X_num)
    X_num_scaled = pd.DataFrame(X_num_scaled, columns=num_features)

    # Concatenate categorical and scaled numerical features
    X = pd.concat([X_cat, X_num_scaled], axis=1)
    X.columns = X.columns.astype(str)  # Convert all feature names to strings
    print("X shape:", X.shape)  # Print the shape of X for debugging

    # Ensure that the number of components in PCA is valid
    num_components = 5  # Match the expected input shape of your neural network model
    print("num_components:", num_components)  # Print the num_components for debugging

    # Handle missing values using SimpleImputer
    imputer = SimpleImputer(strategy='mean')
    X_imputed = imputer.fit_transform(X)

    pca = PCA(n_components=num_components)

    try:
        X_pca = pca.fit_transform(X_imputed)
        print("X_pca shape:", X_pca.shape)  # Print the shape of X_pca for debugging

        # Load your trained neural network model
        json_file = open('model_nn.json', 'r')
        loaded_model_nn = json_file.read()
        json_file.close()

        model_nn = model_from_json(loaded_model_nn)
        model_nn.load_weights('model_nn.h5')
        model_nn.compile(optimizer='adam', loss='binary_crossentropy')

        # Use predict method to get predicted probabilities for the last row only
        predicted_probability = model_nn.predict(X_pca)[0]

        # Determine the prediction result based on the probability
        if predicted_probability >= 0.5:
            prediction_result = 'The customer will churn'
            print('The customer will churn')
        else:
            prediction_result = 'The customer will not churn'
            print('The customer will not churn')

        # Update the resultLabel with the prediction result
        resultLabel.configure(text=prediction_result)

    except ValueError as e:
        print("Error during PCA:", str(e))
        resultLabel.configure(text='Error during PCA. Check console for details.')
    pass
  
        

# Create a button to predict churn and display the result
predict_button = tk.Button(frame, text='Predict Churn', command=predict_and_display_result, height=2, width=20,
                          font=("Arial", 14))
predict_button.grid(row=2, column=1, pady=(10, 20))


# Update the canvas scroll region when the frame size changes
def on_frame_configure(event):
    canvas.configure(scrollregion=canvas.bbox("all"))

frame.bind("<Configure>", on_frame_configure)

transformed_values = []  # Initialize an empty list to store transformed values


# Start the tkinter main loop
window.mainloop()


Transformed values as an array: [1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 2, 1, 1]
userSelection after mapping: [1, 0, 0, 1, 90.0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 2, 1, 1, 25.0, '']
userSelection after edit: [1, 0, 0, 1, 90.0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 2, 1, 1, 25.0, 2250.0, 0, 1, 0.011111111111111112, 3, 0, 24.725274725274726, 0.989010989010989, 6.25, 2250.0, 0]
Length of feature_labels: 19
Length of userSelection: 29
Feature labels: ['Gender', 'SeniorCitizen', 'Partner', 'Dependents', 'Tenure', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges', 'TotalCharges']
User selection: [1, 0, 0, 1, 90.0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 2, 1, 1, 25.0, 2250.0, 0, 1, 0.011111111111111112, 3, 0, 24.725274725274726, 0.989010989010989, 6.25, 2250.0, 0]
Length of feature_labels: 19
Length of userSelection: 29
Feature labels: ['Gender', 'Senior